## Kafka Ingestion

### Required Imports + Subscribing to topic

In [187]:
from confluent_kafka import Consumer
from datetime import datetime
import uuid
import os
from dotenv import load_dotenv
import snowflake.connector
import os
import json
import re
load_dotenv()

KAFKA_SERVER = os.getenv('KAFKA_SERVER')
KAFKA_USERNAME=os.getenv('KAFKA_USERNAME')
KAFKA_PASSWORD=os.getenv('KAFKA_PASSWORD')
KAFKA_TOPIC_NAME = os.getenv('KAFKA_TOPIC_NAME')

values = []

c = Consumer({
    'bootstrap.servers': KAFKA_SERVER,
    'group.id': f'deleton' +str(uuid.uuid1()),
    'security.protocol': 'SASL_SSL',
    'sasl.mechanisms': 'PLAIN',
    'sasl.username': KAFKA_USERNAME,
    'sasl.password': KAFKA_PASSWORD,
    'session.timeout.ms': 6000,
    'heartbeat.interval.ms': 1000,
    'fetch.wait.max.ms': 6000,
    'auto.offset.reset': 'latest',
    'enable.auto.commit': 'false',
    'max.poll.interval.ms': '86400000',
    'topic.metadata.refresh.interval.ms': "-1",
    "client.id": 'id-002-005',
})

c.subscribe([KAFKA_TOPIC_NAME])
"""Kafka consumer ready and consumes messages"""

'Kafka consumer ready and consumes messages'

In [188]:
def extract_values_from_log(string):
    regex=r'\d+.?\d+|\d'
    log_values = re.findall(regex,string)
    return log_values

### Testing Kafka Messages

In [207]:
def wait_for_system_log():
    while True:
        kafka_message = c.poll(0.5)
        if kafka_message is not None:
            log = kafka_message.value().decode('utf-8')

            if 'SYSTEM' in log:
                return kafka_message

In [208]:
i = 0
wait_for_first_user = True #Wait for current user to finish, so can obtain all relevant information

while True:
    kafka_message = c.poll(0.5)

    if wait_for_first_user:
        kafka_message = wait_for_system_log()
        wait_for_first_user = False


    if kafka_message is not None: #exclude none values
        log = kafka_message.value().decode('utf-8')

        if 'SYSTEM' in log:
            system_log = json.loads(log).get('log')
            split_log = system_log.split(' mendoza v9: [SYSTEM] data = ')
            timestamp = split_log[0]
            dictionary_string = split_log[1][:-1]
            user_dictionary = json.loads(dictionary_string)
            print(user_dictionary)

        if 'INFO' in log: #only check for strings with INFO
            values = json.loads(log)
            new_log = values.get('log')
         

            if 'Ride' in new_log: #process strings with Ride info
                new_log = new_log.split(' mendoza v9: [INFO]: Ride - ')
            
                timestamp = new_log[0]
               
                log_values = extract_values_from_log(new_log[1])
                duration=int(float(log_values[0]))
                resistance=log_values[1]
                print(log_values)

            if 'Telemetry' in new_log:
                new_log = new_log.split(' mendoza v9: [INFO]: Telemetry - ')
               
                timestamp = new_log[0]
                log_values = extract_values_from_log(new_log[1])
                heart_rate = log_values[0]
                rpm = log_values[1]
                power= round(float(log_values[2]),3)
                print(log_values)
        


    i += 1

%4|1664968245.204|SESSTMOUT|id-002-005#consumer-2| [thrd:main]: Consumer group session timed out (in join-state steady) after 24571 ms without a successful response from the group coordinator (broker 6, last error was Success): revoking assignment and rejoining group


{'user_id': 4565, 'name': 'Mr Julian Webster', 'gender': 'male', 'address': 'Flat 25p,Walsh shores,Martinport,IG1 1UF', 'date_of_birth': -284601600000, 'email_address': 'mr_j@yahoo.com', 'height_cm': 165, 'weight_kg': 62, 'account_create_date': 1642032000000, 'bike_serial': 'SN0000', 'original_source': 'direct'}
['1.0', '30']
['0', '55', '16.09890976']
['2.0', '30']
['0', '57', '17.00983757']
['3.0', '30']
['0', '57', '17.00983757']
['4.0', '30']
['0', '57', '17.00983757']
['5.0', '30']
['0', '56', '16.55272673']
['6.0', '30']
['0', '53', '15.20140731']
['7.0', '30']
['0', '44', '11.34454484']
['8.0', '30']
['0', '53', '15.20140731']
['9.0', '30']
['0', '56', '16.55272673']
['10.0', '30']
['0', '51', '14.31786487']
['11.0', '30']
['0', '53', '15.20140731']
['12.0', '30']
['0', '56', '16.55272673']
['13.0', '30']
['0', '57', '17.00983757']
['14.0', '30']
['0', '56', '16.55272673']
['15.0', '30']
['83', '54', '15.64844827']
['16.0', '30']
['83', '52', '14.75785543']
['17.0', '30']
['83',

KeyboardInterrupt: 

In [209]:
# system_log = json.loads(system_log).get('log')
split_log = system_log.split(' mendoza v9: [SYSTEM] data = ')
timestamp = split_log[0]
dictionary_string = split_log[1][:-1]
user_dictionary = json.loads(dictionary_string)
user_dictionary

{'user_id': 4565,
 'name': 'Mr Julian Webster',
 'gender': 'male',
 'address': 'Flat 25p,Walsh shores,Martinport,IG1 1UF',
 'date_of_birth': -284601600000,
 'email_address': 'mr_j@yahoo.com',
 'height_cm': 165,
 'weight_kg': 62,
 'account_create_date': 1642032000000,
 'bike_serial': 'SN0000',
 'original_source': 'direct'}

In [210]:
def convert_unix_to_date(unix_timestamp):
    unix_timestamp /= 1000  #convert to seconds as unix timestamp is in milliseconds
    converted_to_date = datetime.utcfromtimestamp(unix_timestamp) 
  
    current_time = datetime.now()
    if converted_to_date > current_time:
        return 'N/A'

    return converted_to_date.strftime('%Y-%m-%d')

user_dictionary['account_create_date'] = convert_unix_to_date(user_dictionary['account_create_date'])
user_dictionary['date_of_birth'] = convert_unix_to_date(user_dictionary['date_of_birth'])

user_dictionary

{'user_id': 4565,
 'name': 'Mr Julian Webster',
 'gender': 'male',
 'address': 'Flat 25p,Walsh shores,Martinport,IG1 1UF',
 'date_of_birth': '1960-12-25',
 'email_address': 'mr_j@yahoo.com',
 'height_cm': 165,
 'weight_kg': 62,
 'account_create_date': '2022-01-13',
 'bike_serial': 'SN0000',
 'original_source': 'direct'}

In [211]:
def split_full_name(name):
    name_list = user_dictionary.get('name').split(' ')
    titles = ['Mr','Mrs','Miss','Ms','Dr']

    if name_list[0] in titles:
        first_name = name_list[1]
        try:
            last_name = name_list[2]
        except IndexError:
            last_name = None

    else:
        first_name = name_list[0]
        last_name = name_list[1]

    return first_name,last_name

first_name,last_name = split_full_name(user_dictionary['name'])

user_dictionary['first_name'] = first_name
user_dictionary['last_name'] = last_name
user_dictionary

{'user_id': 4565,
 'name': 'Mr Julian Webster',
 'gender': 'male',
 'address': 'Flat 25p,Walsh shores,Martinport,IG1 1UF',
 'date_of_birth': '1960-12-25',
 'email_address': 'mr_j@yahoo.com',
 'height_cm': 165,
 'weight_kg': 62,
 'account_create_date': '2022-01-13',
 'bike_serial': 'SN0000',
 'original_source': 'direct',
 'first_name': 'Julian',
 'last_name': 'Webster'}

In [212]:
def flatten_list(address_list):
    flat_list = []

    for element in address_list:
        if type(element) is list:
          
            for item in element:
                flat_list.append(item)
        else:
            flat_list.append(element)
    return flat_list


def split_address(address):
    address_list = address.split(',')

    if len(address_list) < 4:
        address_list[0] = address_list[0].split(' ', 1) 
        address_list = flatten_list(address_list)

    house_number = address_list[0]
    street_name = address_list[1].title()
    region = address_list[2].title()
    postcode = address_list[3]

    return house_number,street_name,region,postcode

house_number,street_name,region,postcode = split_address(user_dictionary['address'])
user_dictionary['house_number'] = house_number
user_dictionary['street_name'] = street_name
user_dictionary['region'] = region
user_dictionary['postcode'] = postcode
user_dictionary['gender'] = user_dictionary['gender'].title()
user_dictionary

    

{'user_id': 4565,
 'name': 'Mr Julian Webster',
 'gender': 'Male',
 'address': 'Flat 25p,Walsh shores,Martinport,IG1 1UF',
 'date_of_birth': '1960-12-25',
 'email_address': 'mr_j@yahoo.com',
 'height_cm': 165,
 'weight_kg': 62,
 'account_create_date': '2022-01-13',
 'bike_serial': 'SN0000',
 'original_source': 'direct',
 'first_name': 'Julian',
 'last_name': 'Webster',
 'house_number': 'Flat 25p',
 'street_name': 'Walsh Shores',
 'region': 'Martinport',
 'postcode': 'IG1 1UF'}

### TODO: 

* Read kafka data into a dataframe (Pandas or PySpark?)
* Write data into Snowflake

In [ ]:
USER = os.environ.get('USER')
ACCOUNT = os.environ.get('ACCOUNT')
PASSWORD = os.environ.get('PASSWORD')
WAREHOUSE= os.environ.get('WAREHOUSE')
DATABASE= os.environ.get('DATABASE')
SCHEMA= os.environ.get('SCHEMA')

In [ ]:
conn = snowflake.connector.connect(
    user=USER,
    password=PASSWORD,
    account=ACCOUNT,
    warehouse=WAREHOUSE,
    database=DATABASE,
    schema='ZOOKEEPERS_BATCH_PRODUCTION'
)
cs = conn.cursor()
"""Already created schema to use"""

'Already created schema to use'

In [ ]:
query = "SHOW SCHEMAS"
result = cs.execute(query)
result.fetchmany(100)

[(datetime.datetime(2022, 10, 5, 2, 38, 0, 876000, tzinfo=<DstTzInfo 'America/Los_Angeles' PDT-1 day, 17:00:00 DST>),
  'EZ_BROKERS',
  'N',
  'N',
  'DELATON',
  'SYSADMIN',
  '',
  '',
  '1'),
 (datetime.datetime(2022, 10, 5, 3, 5, 26, 767000, tzinfo=<DstTzInfo 'America/Los_Angeles' PDT-1 day, 17:00:00 DST>),
  'INFORMATION_SCHEMA',
  'N',
  'N',
  'DELATON',
  '',
  'Views describing the contents of schemas in this database',
  '',
  '1'),
 (datetime.datetime(2022, 9, 29, 1, 9, 6, 401000, tzinfo=<DstTzInfo 'America/Los_Angeles' PDT-1 day, 17:00:00 DST>),
  'PUBLIC',
  'N',
  'N',
  'DELATON',
  'SYSADMIN',
  '',
  '',
  '1'),
 (datetime.datetime(2022, 10, 5, 2, 4, 16, 880000, tzinfo=<DstTzInfo 'America/Los_Angeles' PDT-1 day, 17:00:00 DST>),
  'SNOW_STAGING_ZUCKERBERG',
  'N',
  'N',
  'DELATON',
  'SYSADMIN',
  '',
  '',
  '1'),
 (datetime.datetime(2022, 9, 29, 1, 9, 44, 900000, tzinfo=<DstTzInfo 'America/Los_Angeles' PDT-1 day, 17:00:00 DST>),
  'TEC_BIKE_PRODUCTION',
  'N',
  'N'